# Using `luminol`

In [1]:
import pandas as pd
from luminol.anomaly_detector import AnomalyDetector
from pprint import pprint
from datetime import datetime
from matplotlib import pyplot
from matplotlib import dates

In [8]:
def ingest_csv(fpath):
        # Dateformat: 01-Mar-17 (%d-%b-%y)
        dateparse = lambda x: (pd.to_datetime(pd.datetime.strptime(x, '%d-%b-%y')) - datetime(1970, 1, 1)).total_seconds()
        # nrows=10000
        # df = pd.read_csv(fpath, sep=',', parse_dates=['TRANSACTION_DATE'], date_parser=dateparse)
        df = pd.read_csv(fpath, sep=',')
        df['TRANSACTION_DATE'] = [int(dateparse(x)) for x in df['TRANSACTION_DATE']]
        return df

In [3]:
def plot_anomalies(ts, anomalies):
        # Convert from UNIX timestamps back to regular timestamps
        x_ts_dates = [ datetime.fromtimestamp(x) for x in ts.keys() ]
        x_anomalies_dates = [ datetime.fromtimestamp(x) for x in anomalies.keys() ]

        fig, ax1 = pyplot.subplots()

        # Put data on graph
        pyplot.plot(x_ts_dates, ts.values())
        pyplot.scatter(x_anomalies_dates, anomalies.values(), c='red')

        # Set axis to be labeled by dates
        monthyearFmt = dates.DateFormatter('%Y %B')
        ax1.xaxis.set_major_formatter(monthyearFmt)
        _ = pyplot.xticks(rotation=90)
        pyplot.show()
        pass


In [6]:
def main():
        FILE = 'anomaly-5k.csv'
        # FILE = 'Anomaly_Data_NO-CR.csv'
        df = ingest_csv(FILE)
        ts = {}
        keys = list( df['TRANSACTION_DATE'] )
        values = list(df['DELIVERED_PRICE'])

        #ts = dict(zip(keys, values))
        #ts = df['DELIVERED_PRICE'].groupby(df['TRANSACTION_DATE']).sum()
        ts = df.groupby(['TRANSACTION_DATE'])[ ['DELIVERED_PRICE'] ].sum()
        
        print("keys: ", len(keys))
        print("values: ", len(values))
        print("df.size(): ", df.size)
        print("len(ts.keys())", len(ts.keys()))
        
        pprint(ts)
        return

        detector = AnomalyDetector(ts)
        score = detector.get_all_scores()

        n_anomaly = 0
        anomalies = {}
        for timestamp, value in score.iteritems():
                if value > 2:
                        n_anomaly += 1
                        print(timestamp, value)
                        anomalies[timestamp] = ts[timestamp]
                # print(timestamp, value)

        print("n_anomaly:", n_anomaly, "of", len(df), "(", (n_anomaly/len(df))*100, "%)")

        for anomaly in detector.get_anomalies():
                pprint( vars(anomaly) )

        print(len(detector.get_anomalies()))

        plot_anomalies(ts, anomalies)

In [9]:

#if __name__ == "__main__":
#        main()
main()

keys:  5363
values:  5363
df.size():  69719
len(ts.keys()) 1
                  DELIVERED_PRICE
TRANSACTION_DATE                 
1488326400                 322.15
1488499200                 928.88
1488758400                 736.05
1488931200                 295.87
1489104000                 970.93
1489363200                 474.23
1489536000                 470.67
1489622400                 -40.62
1489708800                1020.03
1489968000                 403.63
1490140800                 379.24
1490313600                 935.01
1490572800                 330.09
1490745600                 439.02
1490918400                 978.53
1491177600                 442.73
1491264000                  29.55
1491350400                 427.95
1491523200                1001.04
1491782400                 525.45
1491955200                 459.98
1492128000                 859.37
1492387200                 603.70
1492473600                 -19.99
1492560000                 498.90
1492732800           